# Week4 Modeling

In [3]:
import pandas as pd
import numpy as np

#### Using pandas with large data
https://www.dataquest.io/blog/pandas-big-data/
1. downcasting numeric columns to more efficient types
2. converting string columns to the categorical type



## import all files

메모리를 아끼기 위해 np.int8등 설정해줌

In [4]:
#import all files
aisles = pd.read_csv("aisles.csv")
departments = pd.read_csv("departments.csv")
order_products_prior = pd.read_csv("order_products__prior.csv",
                    dtype={'order_id': np.int32, 
                      'product_id': np.int32, 
                      'add_to_cart_order': np.int16, 
                      'reordered': np.int8})
# prior : 마지막 주문을 제외한 모든 주문
order_products_train = pd.read_csv("order_products__train.csv",
                     dtype={'order_id': np.int32, 
                      'product_id': np.int32, 
                      'add_to_cart_order': np.int16, 
                      'reordered': np.int8})
# train : 마지막 주문
orders = pd.read_csv("orders.csv",
                    dtype={'order_id': np.int32, 
                       'user_id': np.int32, 
                       'order_number': np.int32, 
                       'order_dow': np.int8, 
                       'order_hour_of_day': np.int8, 
                       'days_since_prior_order': np.float16})
products = pd.read_csv("products.csv")
sample_submission = pd.read_csv("sample_submission.csv")

orders의 eval_set을 integer로 바꾸자
<br>(memory consumption을 줄여준다고 함)

prior : 0, train : 1, test : 2

In [5]:
orders.eval_set = orders.eval_set.replace({'prior': 0, 'train': 1, 'test':2}).astype(np.int8)

## Data 합치기

### goods = products + departments + aisles

In [6]:
goods = pd.merge(left=pd.merge(left=products, right=departments, how='left'), right=aisles, how='left')

In [7]:
goods.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49688 entries, 0 to 49687
Data columns (total 6 columns):
product_id       49688 non-null int64
product_name     49688 non-null object
aisle_id         49688 non-null int64
department_id    49688 non-null int64
department       49688 non-null object
aisle            49688 non-null object
dtypes: int64(3), object(3)
memory usage: 2.7+ MB


### order_details 만들기

#### order_products_prior + order_products_train

In [8]:
order_details= pd.concat([order_products_prior, order_products_train], axis = 0)

In [9]:
order_details.head(3)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0


#### + orders

In [10]:
from functools import partial

In [11]:
order_details = pd.merge(
    left = order_details,
    right = orders,
    how ="left",
    on="order_id").apply(partial(pd.to_numeric, errors='ignore', downcast='integer'))


/opt/conda/lib/python3.6/site-packages/numpy/core/numeric.py:2549: RuntimeWarning: overflow encountered in subtract
  result = less_equal(abs(x-y), atol + rtol * abs(y))


In [12]:
order_details.head(3)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,0,3,5,9,8.0
1,2,28985,2,1,202279,0,3,5,9,8.0
2,2,9327,3,0,202279,0,3,5,9,8.0


#### + goods

In [13]:
goods.head(2)

,product_id,product_name,aisle_id,department_id,department,aisle
0,1,Chocolate Sandwich Cookies,61,19,snacks,cookies cakes
1,2,All-Seasons Salt,104,13,pantry,spices seasonings


In [14]:
order_details = pd.merge(
    left = order_details,
    right = goods[['product_id','aisle_id','department_id']],
    how ="left",
    on="product_id").apply(partial(pd.to_numeric, errors='ignore', downcast='integer'))

/opt/conda/lib/python3.6/site-packages/numpy/core/numeric.py:2549: RuntimeWarning: overflow encountered in subtract
  result = less_equal(abs(x-y), atol + rtol * abs(y))


In [15]:
print(len(order_details))
order_details.head(3)

33819106


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id
0,2,33120,1,1,202279,0,3,5,9,8.0,86,16
1,2,28985,2,1,202279,0,3,5,9,8.0,83,4
2,2,9327,3,0,202279,0,3,5,9,8.0,104,13


In [16]:
print('Memory consumption: {:.2f} Mb'.format(sum(order_details.memory_usage(index=True, deep=True) / 2**20)))

Memory consumption: 1032.08 Mb


downcast를 사용하고 나서 전체 memory consumption이 
<br>1548.12Mb에서 1032.08Mb로 감소됨 

---
## 파생변수 만들기

### user에 관한 변수들

user가 instacart 사용한지 얼마나 되었는지 user_period

In [17]:
users = orders[orders['eval_set'] == 0].groupby(['user_id'])['order_number'].max().to_frame('user_orders')

In [18]:
users['user_period'] = orders[orders['eval_set']==0].groupby(['user_id'])['days_since_prior_order'].sum()

user_days_since_prior_mean

In [19]:
users['user_days_since_prior_mean'] = orders[orders['eval_set'] == 0].groupby(['user_id'])['days_since_prior_order'].mean()

user의 재구매 비율 user_reorder_ratio 

In [20]:
users['user_reorder_ratio'] = order_details[order_details.eval_set==0].groupby('user_id')['reordered'].mean()

user가 여태 산 총 상품 종류 수 user_products_unique

In [21]:
users['user_products_unique'] = order_details[order_details.eval_set==0].groupby('user_id')['product_id'].nunique()

user가 보통 사는 상품개수 user_average_product_num

In [22]:
user_total_product_num = order_details[order_details.eval_set==0].groupby('user_id').size().to_frame('user_total_product_num')
user_order_num = orders[orders['eval_set'] == 0].groupby(['user_id'])['order_number'].max().to_frame('user_order_num')
users['user_average_product_num'] = user_total_product_num['user_total_product_num']/user_order_num['user_order_num']

In [23]:
users.head(3)

,user_orders,user_period,user_days_since_prior_mean,user_reorder_ratio,user_products_unique,user_average_product_num
user_id,,,,,,
1,10,176.0,19.562500,0.694915,18,5.900000
2,14,198.0,15.234375,0.476923,102,13.928571
3,12,133.0,12.093750,0.625000,33,7.333333


### product 에 관한 새로운 변수들
product별 평균적 재구매 비율을 구함

In [24]:
prod = order_details

In [25]:
prod_temp = order_details.groupby('product_id').mean()['reordered'].reset_index()
prod_temp['prod_reordered'] = prod_temp['reordered']
prod_temp = prod_temp.drop('reordered', axis = 1)

In [26]:
prod_temp.head(2)

,product_id,prod_reordered
0,1,0.614627
1,2,0.138298


In [27]:
prod = prod.merge(prod_temp, on="product_id")

In [28]:
prod.head(2)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id,prod_reordered
0,2,33120,1,1,202279,0,3,5,9,8.0,86,16,0.71024
1,26,33120,5,0,153404,0,2,0,16,7.0,86,16,0.71024


#### user 부분 merge

In [29]:
users_temp = users.reset_index()
users_temp.head(2)

,user_id,user_orders,user_period,user_days_since_prior_mean,user_reorder_ratio,user_products_unique,user_average_product_num
0,1,10,176.0,19.562500,0.694915,18,5.900000
1,2,14,198.0,15.234375,0.476923,102,13.928571


In [30]:
users_temp.columns

Index(['user_id', 'user_orders', 'user_period', 'user_days_since_prior_mean',
       'user_reorder_ratio', 'user_products_unique',
       'user_average_product_num'],
      dtype='object')

In [31]:
order_details = order_details.merge(users_temp[['user_id', 'user_period', 'user_days_since_prior_mean',
       'user_reorder_ratio', 'user_products_unique',
       'user_average_product_num']], on='user_id')

#### product 부분 merge

In [32]:
order_details = order_details.merge(prod_temp, on="product_id")

In [33]:
order_details.head(2)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id,user_period,user_days_since_prior_mean,user_reorder_ratio,user_products_unique,user_average_product_num,prod_reordered
0,2,33120,1,1,202279,0,3,5,9,8.0,86,16,151.0,21.578125,0.477778,47,11.25,0.71024
1,104690,33120,2,1,202279,0,8,6,10,30.0,86,16,151.0,21.578125,0.477778,47,11.25,0.71024


In [34]:
order_details.to_csv("order_details.csv")

KeyboardInterrupt: 

---
## Train/Test set

In [35]:
train = order_details[order_details.eval_set!=2]

In [36]:
train.head(3)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id,user_period,user_days_since_prior_mean,user_reorder_ratio,user_products_unique,user_average_product_num,prod_reordered
0,2,33120,1,1,202279,0,3,5,9,8.0,86,16,151.0,21.578125,0.477778,47,11.25,0.71024
1,104690,33120,2,1,202279,0,8,6,10,30.0,86,16,151.0,21.578125,0.477778,47,11.25,0.71024
2,132412,33120,1,1,202279,0,6,5,9,30.0,86,16,151.0,21.578125,0.477778,47,11.25,0.71024


In [37]:
test = orders[orders.eval_set == 2]

---
## Lightgbm

In [38]:
from sklearn import preprocessing
from sklearn.cross_validation import cross_val_score, train_test_split

# Models for Classification
from lightgbm import LGBMClassifier

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### 데이터 불러오기

In [39]:
len(train.columns)

18

In [40]:
len(train.columns) == len(test.columns)

False

In [41]:
features = [
 'order_number',
 'order_dow',
 'order_hour_of_day',
 'aisle_id',
 'department_id',
 'prod_reordered',
 'user_period',
 'user_days_since_prior_mean',
 'user_reorder_ratio',
 'user_products_unique',
 'user_average_product_num']

#### Score Function 정의

In [42]:
from sklearn.metrics import auc, f1_score, accuracy_score

In [43]:
train_subset = train[train.index % 10 != 0]
valid_subset = train[train.index % 10 == 0]

X_train = train_subset[features]
y_train = train_subset.reordered
X_val = valid_subset[features]
y_val = valid_subset.reordered

In [44]:
lgb = LGBMClassifier()
lgb.fit(X_train,y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)

점수 확인

In [45]:
y_pred = lgb.predict(X_val)
print("here5")
predictions = [round(value) for value in y_pred]
print("here6")
# evaluate predictions
accuracy = accuracy_score(y_val, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

here5
here6
Accuracy: 75.75%


그냥 돌렸을때 73.43% 나옴

feature몇개 추가하고나서 77.46% 흠

## 결과내기

### 고객이 전에 구입했던 모든 상품들 중에서 예측을 해보겠음

#### + 전에 구입한 모든 상품을 후보로 올려놓음

In [46]:
len(train.columns)

18

In [47]:
test_by_history = (order_details[(order_details.user_id
        .isin(test.user_id))]
        .groupby(['user_id','product_id']).size())

In [48]:
test_by_history = test_by_history.reset_index().drop(0, axis = 1)

In [49]:
test_by_history.head(2)

,user_id,product_id
0,3,248
1,3,1005


#### + orders안에있던 정보들

In [50]:
test_by_history2 = test_by_history.merge(test, on="user_id")

#### + order_details 에 있던 정보들
##### user에 관한것

In [51]:
users = users.reset_index()

In [52]:
test_by_history3 = test_by_history2.merge(users[['user_id','user_period', 'user_days_since_prior_mean', 'user_reorder_ratio',
       'user_products_unique', 'user_average_product_num']], on="user_id")

##### product에 관한것

In [53]:
prod_info = prod[['product_id','aisle_id', 'department_id', 'prod_reordered']].drop_duplicates(['product_id'], keep='last')

In [54]:
test_by_history4 = test_by_history3.merge(prod_info[['product_id','aisle_id', 'department_id', 'prod_reordered']], on="product_id")

In [57]:
test_by_history4.head(2)

,user_id,product_id,order_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_period,user_days_since_prior_mean,user_reorder_ratio,user_products_unique,user_average_product_num,aisle_id,department_id,prod_reordered
0,3,248,2774568,2,13,5,15,11.0,133.0,12.09375,0.625000,33,7.333333,117,19,0.401366
1,12,248,1356845,2,6,1,20,30.0,100.0,25.00000,0.175676,61,14.800000,117,19,0.401366


test_by_history4에는 reordered와 days_since_prior_order정보가 없음

### 전체 train-set fitting 시키기

In [132]:
features = [
 'order_number',
 'order_dow',
 'order_hour_of_day',
 'aisle_id',
 'department_id',
 'prod_reordered',
 'user_period',
 'user_days_since_prior_mean',
 'user_reorder_ratio',
 'user_products_unique',
 'user_average_product_num']

In [ ]:
X_train_all = train[features]
y_train_all = train.reordered

In [ ]:
lgb_final = LGBMClassifier()
lgb_final.fit(X_train_all, y_train_all)

In [61]:
test_by_history4['reordered'] = lgb_final.predict(test_by_history4[features])

In [68]:
result_frame = test_by_history4[['order_id','product_id','reordered']]

In [69]:
result_frame_1 = result_frame[result_frame.reordered == 1]

In [70]:
result_frame01 = test[['order_id']].merge(result_frame_1, on="order_id", how="outer")

In [71]:
result_frame01['reordered'] = result_frame01['reordered'].fillna(0)

In [72]:
result_frame01['product_id'] = [x.astype(np.int64) for x in result_frame01['product_id'] ]

In [73]:
result_frame01.groupby('order_id').size().mean()

42.221373333333332

In [74]:
order_details[order_details.eval_set!=2].groupby('order_id').size().mean()

10.10707325550502

음? 왜케 많이 주문한다고 나왔지?

In [75]:
result_csv = result_frame01.groupby('order_id')['product_id'].apply(lambda x: ' '.join([str(e) for e in set(x)])).reset_index()

In [76]:
result_csv.columns = ['order_id','products']

In [77]:
result_csv.head()

,order_id,products
0,17,31964 21709 47766 21903
1,34,26369 36994 46979 20995 7559 48523 39180 651 5...
2,137,21903 49683 24852 2326 29594 16797 2078 18465 ...
3,182,11520 17794 40706 12036 34824 38281 44683 2190...
4,257,1025 24964 11140 24838 11782 29837 37646 39055...


In [78]:
result_csv.to_csv('0227-1.csv', encoding='utf-8', index=False)

In [128]:
result_csv[result_csv.order_id == 4065]

,order_id,products
74,4065,-9223372036854775808


In [131]:
float(-9223372036854775808)

-9.223372036854776e+18

## 바로전이랑 똑같이 주문했을때

바로 전이랑 똑같이 주문한다 가정

In [117]:
same_last = order_details[(order_details.eval_set==2)]

In [118]:
last_orders = same_last.groupby('user_id')['order_number'].max()


In [126]:
test_history = order_details[(order_details.eval_set!=2)]
last_orders = test_history.groupby('user_id')['order_number'].max()

def get_last_orders():
    t = pd.merge(
            left=pd.merge(
                    left=last_orders.reset_index(),
                    right=test_history,
                    how='inner',
                    on=['user_id', 'order_number']
                )[['user_id', 'product_id']],
            right=order_details[order_details.eval_set!=2][['user_id', 'order_id']],
            how='left',
            on='user_id'
        ).groupby('order_id')['product_id'].apply(lambda x: ' '.join([str(e) for e in set(x)])).reset_index()
    t.columns = ['order_id', 'products']
    return t

# save submission
get_last_orders().to_csv('less_dumb_subm_last_order.csv', encoding='utf-8', index=False)

KeyboardInterrupt: 

In [393]:
temp = pd.read_csv('same_last_order.csv')

In [397]:
temp

,order_id,products
0,17,13107 26429 39275 21463
1,34,2596 49191 44632 44912 47792 16083 23028 39475...
2,137,25890 44422 13740 34126 5134 23794 24852 2326 ...
3,182,10243 5479 33000 10025 37642 39275 40377 32109...
4,257,27104 11140 11782 29837 37646 2063 49235 24852...
5,313,12779 13198 21903 45007 28535 45646 30391 46906
6,353,33000 35561 40688 21137 17302 48183
7,386,15872 260 38281 45066 39180 24852 35221 47766 ...
8,414,27705 20564 36865 31215
9,418,27267 13702 40268 5262 35918 21454 44910 1970 ...
